In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg as LA

Create a single wave packet for any system size $N_s$, position center $x^\text{(mean)}$, momentum center $n_k^\text{(mean)}$, and width $\sigma_k$. The $\beta$'s cancel out the phases of the coefficients and $\theta$'s diagonlize row by row the coefficients. To create a second wave packet, run the notebook again with a new position center and momentum center.

In [ ]:
n_s = 3 # subsystem
N_s = 8

In [20]:
x_mean = 1 # Choose a position center
nk_mean = -3 # Choose a momentum center
sig_k = 3/2 # Choose the width of wave packet in momentum space
nks = np.arange(-N_s + 1, N_s + 1, 2)

In [12]:
# An alternative definition of a wave packet:
## - Make packet in momentum space
## - Make packet in position space
# def phi_mom(x_mean, nk_mean, nk):
#     k = nk * np.pi / N_s
#     k_mean = nk_mean * np.pi / N_s
#     return np.exp(1j * k * x_mean) * np.exp(-(k - k_mean)**2 / sig_k**2)

# def phi_pos(x_mean, x, nk_mean):
#     phi_moms = np.zeros(N_s, dtype=complex)
#     for j in range(N_s):
#         phi_moms[j] = phi_mom(x_mean, nk_mean, nks[j])
#     phi_moms /= np.linalg.norm(phi_moms)
#     phi = 0
#     for j in range(N_s):
#         phi += phi_moms[j] * np.exp(-1j * nks[j] * np.pi / N_s * x)
#     return 1/np.sqrt(N_s) * phi

x_dif = [-1, 0, 1]
# x_dif = np.roll(np.array(x_dif),  4)
def GWP_coeff(x, x_dif, nk_mean, sig_k):
    k = nk_mean * np.pi / N_s
    coeff = 1 / np.sqrt(N_s) * np.exp(-1j * k * x)
    coeff *= np.exp(-x_dif[x]**2 / sig_k**2)
    return coeff

In [13]:
GWP_coeffs = []
for i in range(n_s):
    coeff = GWP_coeff(i, x_dif, nk_mean, sig_k)
    GWP_coeffs.append(coeff)
GWP_coeffs = np.array(GWP_coeffs)
GWP_coeffs /= np.linalg.norm(GWP_coeffs)

betas = []
for i in range(n_s):
    betas.append(-np.angle(GWP_coeffs[i]))
betas = np.array(betas)
p = np.diag(np.exp(1j * betas))

In [14]:
def Giv_Mat(theta, s):
    mat = np.identity(n_s, dtype=float)
    mat[s-1, s-1] = np.cos(theta)
    mat[s-1, s] = -np.sin(theta)
    mat[s, s-1] = np.sin(theta)
    mat[s, s] = np.cos(theta)
    return mat

In [15]:
GWP_coeffs = p @ GWP_coeffs
thetas = []
for i in range(n_s-1):
    j = n_s - 1 - i
    theta = np.arctan(-np.abs(GWP_coeffs[j]) / np.abs(GWP_coeffs[j-1]))
    thetas.append(theta)
    GWP_coeffs = Giv_Mat(theta, j) @ GWP_coeffs

In [16]:
GWP_coeffs # Verifies we diagonlized vector
# First element should be 1, and zero for all
# other elements.

array([ 1.00000000e+00-2.10379795e-17j,  2.62617151e-17-1.13554270e-17j,
       -4.76375867e-17+1.12028132e-17j])

In [17]:
thetas # Givens Rotations Angles

[-0.5701501348339028, -1.0758501050425324]

In [18]:
betas # Negative of phase angles

array([-0.        , -1.17809725, -2.35619449])